In [1]:
!pip install -q  fastai==1.0.61

In [2]:
# check version of fastai
import fastai

fastai.__version__

'1.0.61'

In [3]:
# check version of fastai
import fastai
fastai.__version__

'1.0.61'

In [4]:
import numpy as np 
import pandas as pd 

from fastai import *
from fastai.vision import *
from pathlib import Path
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
# For jupyter notebook to see the config/paths.py file
import sys
sys.path.append('../config/')

from paths import train_path, test_path, data_dir, sample_submission_path

### tfms helps to do data augmentation on the data to have a better pixel values

In [6]:
tfms=get_transforms(do_flip=True,flip_vert=True,max_lighting=0.3,max_zoom=1.8,max_warp=0.2,max_rotate=45)

### Read the data and pass the data augmentation function

In [7]:
image_path = data_dir/"images"

In [8]:
df = pd.read_csv(train_path)

In [9]:
data = ImageDataBunch.from_df(image_path,
                              df,
                              size=224,
                              valid_pct=0.20,
                              ds_tfms=tfms,
                              num_workers=2,
                              bs=8,
                             suffix='.jpg').normalize(imagenet_stats)

In [10]:
# There are 9 classes!
print(data.c)

9


In [11]:
# Look at some examples
# data.show_batch(rows=3, figsize=(15, 15)) 

### Create a baseline model using resnet18

In [12]:
learn = cnn_learner(data, models.resnet18, metrics=[error_rate, accuracy])

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\alifa/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [13]:
# Fit
num_epochs = 3
learn.fit_one_cycle(num_epochs, 1e-4)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,3.093541,2.068223,0.708567,0.291433,08:27
1,2.583602,1.644255,0.562850,0.437150,08:07
2,2.503435,1.604768,0.554844,0.445156,08:35


## predict on test set

In [14]:
test_df = pd.read_csv(test_path)

In [15]:
test_df.head()

,img_IDS
0,ImageID_USRB8QNG
1,ImageID_SZ8D1ZJI
2,ImageID_4OJO2F8J
3,ImageID_IEE4XV0B
4,ImageID_BP0O0WZ9


In [16]:
test = ImageList.from_df(test_df, image_path, suffix='.jpg')

In [17]:
data.add_test(test)

In [21]:
preds_test,y_test = learn.get_preds(DatasetType.Test)

In [43]:
preds_test[:, 0]  # Church predictions

tensor([0.0284, 0.0537, 0.2296,  ..., 0.0530, 0.1062, 0.0526])

In [28]:
y_test

tensor([0, 0, 0,  ..., 0, 0, 0])

In [35]:
sub=pd.read_csv(sample_submission_path)
k=[]
for col in sub.columns: 
  k.append(col) # creating list of the label

In [36]:
k

['img_IDS',
 'Church',
 'Enough/Satisfied',
 'Friend',
 'Love',
 'Me',
 'Mosque',
 'Seat',
 'Temple',
 'You']

In [37]:
import os
submission = pd.DataFrame()
submission["ID"] = test_df["img_IDS"]
for i, c in enumerate(learn.data.classes):
  print(c)
  submission[c] = preds_test[:,i]
submission.head()

Church
Enough/Satisfied
Friend
Love
Me
Mosque
Seat
Temple
You


,ID,Church,Enough/Satisfied,Friend,Love,Me,Mosque,Seat,Temple,You
0,ImageID_USRB8QNG,0.028378,0.056988,0.031725,0.018164,0.121713,0.053303,0.326470,0.094763,0.268495
1,ImageID_SZ8D1ZJI,0.053656,0.798165,0.010558,0.001678,0.005340,0.091308,0.000542,0.028016,0.010736
2,ImageID_4OJO2F8J,0.229554,0.267089,0.014751,0.012230,0.041698,0.380654,0.022075,0.026446,0.005503
3,ImageID_IEE4XV0B,0.080038,0.017763,0.002533,0.040723,0.170535,0.069552,0.407223,0.019221,0.192412
4,ImageID_BP0O0WZ9,0.037722,0.014850,0.007672,0.042284,0.006422,0.037195,0.104518,0.385877,0.363461


In [38]:
submission.to_csv('baseline_model.csv', index=False)

## Improving your Model Accuracy

* Perform more data augmentation
* Try other pre-trained model like (efficientnet model)
* Improve the epochs size
* Test Time Augmentations 